In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [19]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        # LSTM for encoding the time series
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Linear layers to get the mean and log variance for the latent variables
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
    
    def forward(self, x):
        # x shape: (batch_size, T, N)
        lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch_size, T, hidden_dim)
        
        # Compute mean and log variance for each time step
        mu = self.fc_mu(lstm_out)  # mu shape: (batch_size, T, latent_dim)
        logvar = self.fc_logvar(lstm_out)  # logvar shape: (batch_size, T, latent_dim)
        
        return mu, logvar

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, num_layers=1):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        
        # LSTM for decoding the latent sequence
        self.lstm = nn.LSTM(latent_dim, hidden_dim, num_layers, batch_first=True)
        
        # Linear layer to map hidden state back to original time-series space
        self.fc_out = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, z):
        # z shape: (batch_size, T, latent_dim)
        lstm_out, _ = self.lstm(z)  # lstm_out shape: (batch_size, T, hidden_dim)
        
        # Map the LSTM output to the original time-series space
        out = self.fc_out(lstm_out)  # out shape: (batch_size, T, output_dim)
        
        return out

In [21]:
class DVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(DVAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim, num_layers)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim, num_layers)
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar

def loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld_loss

In [22]:
# Define model and optimizer
input_dim = 10  # Number of input variables (N)
latent_dim = 5  # Number of latent variables (M)
hidden_dim = 20  # Hidden dimension for LSTM
num_layers = 1  # Number of LSTM layers
model = DVAE(input_dim, hidden_dim, latent_dim, num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # x shape: (batch_size, T, N)
    x = torch.randn(32, 100, input_dim)  # Example batch of data
    recon_x, mu, logvar = model(x)
    loss = loss_function(recon_x, x, mu, logvar)
    
    loss.backward()
    optimizer.step()
    
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

Epoch 1, Loss: 33462.01953125
Epoch 2, Loss: 33585.53515625
Epoch 3, Loss: 33324.5546875
Epoch 4, Loss: 33436.4609375
Epoch 5, Loss: 33156.2421875
Epoch 6, Loss: 33376.2578125
Epoch 7, Loss: 33095.4921875
Epoch 8, Loss: 33099.81640625
Epoch 9, Loss: 32776.83984375
Epoch 10, Loss: 32631.515625
Epoch 11, Loss: 32534.275390625
Epoch 12, Loss: 32829.953125
Epoch 13, Loss: 32631.98828125
Epoch 14, Loss: 32438.240234375
Epoch 15, Loss: 32444.00390625
Epoch 16, Loss: 32660.650390625
Epoch 17, Loss: 33054.6875
Epoch 18, Loss: 33018.58203125
Epoch 19, Loss: 32296.109375
Epoch 20, Loss: 32450.732421875
Epoch 21, Loss: 32406.451171875
Epoch 22, Loss: 32282.8046875
Epoch 23, Loss: 32967.55859375
Epoch 24, Loss: 32566.810546875
Epoch 25, Loss: 32645.82421875
Epoch 26, Loss: 32531.9375
Epoch 27, Loss: 32402.64453125
Epoch 28, Loss: 31625.0859375
Epoch 29, Loss: 32064.599609375
Epoch 30, Loss: 32421.73046875
Epoch 31, Loss: 32504.478515625
Epoch 32, Loss: 31873.455078125
Epoch 33, Loss: 32313.1425781